In [8]:
import torch
torch.__version__

'1.6.0'

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() 
                      else "cpu")
print(device)

cuda:0


# Data generation

In [10]:
import numpy as np
import matplotlib.pyplot as plt
def function_2d(x,y):
    ### shape ### x,y:(num_ins,1) 
    
    term_sqrt = np.sqrt(x*x+y*y)
    term1 = np.sin(20*term_sqrt)/(20*term_sqrt)
    term2 = (1/5)*np.cos(10*term_sqrt)
    term3 = y/2 - 0.3
    
    label = term1 + term2 + term3
    
    return label
    ###shape### label : (num_ins,1)

# generate dataset
def generate_data(dim):
    
    #generate 100x100 points
    x = np.linspace(-1,1,dim)
    y = np.linspace(-1,1,dim)
    xx,yy = np.meshgrid(x,y)
    ###shape### xx,yy : (dim,dim)
    zz = function_2d(xx,yy)
    ###shape### zz : (dim,dim)
    
    # add noise
    noise = np.random.standard_normal((dim,dim))
    zz = zz + 0.1*noise
    
    # create dataset
    data1 = xx.reshape(-1,1)
    data2 = yy.reshape(-1,1)
    label = zz.reshape(-1,1)
    dataset = np.hstack((data1,data2))
    dataset = np.hstack(((dataset,label)))
    ###shape### dataset : (dim*dim,3) np.array
    return torch.Tensor(dataset)

dataset= generate_data(100)

In [11]:
dataset[0][0:2]

tensor([-1., -1.])

In [12]:
from sklearn.model_selection import train_test_split
trainset, validset = train_test_split(dataset, test_size=0.3, random_state=0)
validset, testset = train_test_split(validset,test_size=0.5,random_state=0)

# Define Network

In [13]:
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

#ngpu = 1
#hidden_neurons = 16
class MLP(nn.Module):
    def __init__(self, hidden_neurons):
        super(MLP, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.Linear(2,hidden_neurons),
            #nn.ReLU(True),
            nn.Sigmoid(),
            nn.Linear(hidden_neurons,1),
            
           # nn.Dropout(0.2),
           # nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(-1,2)
        return self.main(x)
    
def weights_init(m):
    if isinstance(m, nn.Linear):
        m.weight.data.normal_(0.0,1.0)
        torch.nn.init.zeros_(m.bias.data)
        
#model=MLP(ngpu)#.to(device)
#model.apply(weights_init)

In [14]:
# print("Generator's state_dict:\n")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t ", model.state_dict()[param_tensor].size())

In [15]:
# from torchsummary import summary
# summary(model, (1,1,2))

# Loss & Optimizer

In [16]:
from torch.autograd import Variable
def rmse(output ,label):
    sum0 = 0
    for i in range(0,1500):
        
        term1 = float(output[i])-float(label[i])
        term2 = term1 * term1
        sum0 = sum0 + term2
        
    result = np.sqrt(sum0/1500)
    #result = torch.Tensor(result)
    result = torch.from_numpy(np.array(result))
    result = Variable(result,requires_grad=True)
    return result
   # return torch.from_numpy(result)
    
criterian = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3000, gamma=0.5)

NameError: name 'model' is not defined

# Define Train Validation and Test

In [ ]:
def train(epoch):
    #print('epoch : ',epoch+1)
    model.train() 

    inputs = Variable(trainset[:,0:2]).view(7000, 2)#.to(device)
    labels = Variable(trainset[:,2]).view(7000, 1)#.to(device)
    
    optimizer.zero_grad()

    outputs = model(inputs)
   # train_out.append(outputs)

    loss = criterian(outputs,labels)
#     if(epoch%5000==0 or epoch==num_epochs-1):
#         print(epoch,loss.item())
    loss.backward(retain_graph=True)
    optimizer.step()
    scheduler.step()
    
def valid(epoch):
    model.eval()
    with torch.no_grad():

        inputs = Variable(validset[:,0:2]).view(1500, 2)#.to(device)
        labels = Variable(validset[:,2]).view(1500, 1)#.to(device)

        outputs = model(inputs)

        loss = rmse(outputs,labels)

        if(epoch%3000==0 or epoch==num_epochs-1):
            print(epoch,'valid: Loss: {:.4f}'.format(loss.item()))      
    #scheduler.step()      
    return loss.item()
    
def test(epoch):
    model.eval()
    with torch.no_grad():

        inputs = Variable(testset[:,0:2]).view(1500, 2)#.to(device)
        labels = Variable(testset[:,2]).view(1500, 1)#.to(device)

        outputs = model(inputs)

        loss = rmse(outputs,labels)

        if(epoch%3000==0 or epoch==num_epochs-1):
            print(epoch,'test: Loss: {:.4f}'.format(loss.item()))
    #scheduler.step()    
    return loss.item()

# Nested CV

In [ ]:
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

num_epochs = 10000
Err_val=[]  
h=[16,24,32,40,48,56]
dataset= generate_data(100)
for fold in range(0,3):
    dataset_shuffled=dataset[torch.randperm(len(dataset))]
    valid_n_test=dataset_shuffled[3000*fold:3000*(fold+1)][:]
    validset = valid_n_test[0:1500][:]
    
    if(fold == 0):
        trainset = dataset_shuffled[3000:10000][:]
    elif(fold == 1):
        t1 = dataset_shuffled[0:3000][:]
        t2 = dataset_shuffled[6000:10000][:]
        t = np.vstack((t1,t2))
        trainset = torch.Tensor(t)
    else:
        t1 = dataset_shuffled[0:6000][:]
        t2 = dataset_shuffled[9000:10000][:]
        t = np.vstack((t1,t2))
        trainset = torch.Tensor(t) 
    
    for i in range(0,len(h)):
        print('fold =',fold,', hidden neurons =',h[i])
        model=MLP(h[i])
        model.apply(weights_init)
        for epoch in range(0,num_epochs):
            train(epoch)
            a = valid(epoch)
        Err_val.append(a)

In [ ]:
Err_val